In [2]:
import os
os.environ["DDE_BACKEND"] = "pytorch"  # jak w treningu

import deepxde as dde
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# ------- 1) IDENTYCZNY transform jak w treningu -------
stretch = 2.5
def transform(x, y):
    X1 = x[:, 0:1]; X2 = x[:, 1:2]; t  = x[:, 2:3]
    ratio = (stretch - 1.0) * t + 1
    new_x1 = (X1 + 1.0) * (X1 - 1.0) * y[:, 0:1] + (X1 + 1.0) * ratio - 1.0
    new_x2 = (X1 + 1.0) * (X1 - 1.0) * y[:, 1:2] + X2
    return torch.cat([new_x1, new_x2, y[:, 2:3]], dim=1)

# ------- 2) Minimalne "dummy" dane tylko do zbudowania Model -------
outer = dde.geometry.geometry_2d.Rectangle([-1.0, -0.5], [1.0, 0.5])
time  = dde.geometry.TimeDomain(0.0, 1.0)
dom   = dde.geometry.GeometryXTime(outer, time)

def pde_dummy(x, y):
    z = y[:, 0:1] * 0
    return [z, z, z]  # 3 składowe jak w treningu

data = dde.data.TimePDE(dom, pde_dummy, [], num_domain=1, num_boundary=0, num_initial=0, num_test=1)

# ------- 3) TAKA SAMA architektura sieci jak w treningu -------
net = dde.maps.FNN([3] + [30]*3 + [3], "tanh", "Glorot uniform")
net.apply_output_transform(transform)

# Zmienne zewnętrzne (żeby wczytać ich stan); nazwy jak w treningu
mu = dde.Variable(4.0)
lam_load = dde.Variable(7.0)

# Loss weights są do predykcji nieistotne, ale muszą istnieć
weights = [1] * 7

model = dde.Model(data, net)
model.compile("adam", lr=1e-3,
              external_trainable_variables=[mu, lam_load],
              loss_weights=weights)

# ------- 4) Wczytaj checkpoint -------
CKPT_DIR = "Siyuan_AUG_12-200000.pt"   # <-- podmień jeśli zapisałeś pod inną nazwą/ścieżką
model.restore(CKPT_DIR, verbose=1)

Compiling model...
'compile' took 0.000107 s

Restoring model from Siyuan_AUG_12-200000.pt ...



In [5]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from collections import Counter

X = np.load("train_x.npy")  # (N,3): x,y,t
assert X.shape[1] == 3

# policz ile punktów ma każdy czas
t_vals, counts = np.unique(X[:,2], return_counts=True)
t2n = dict(zip(t_vals, counts))

# wybierz 5 czasów z największą liczbą punktów (lub ile chcesz)
top_k = 5
picked_ts = [t for t, _ in sorted(t2n.items(), key=lambda kv: kv[1], reverse=True)[:top_k]]
print("Wybrane czasy (najgęstsze):", picked_ts, " | liczności:", [t2n[t] for t in picked_ts])

outdir = Path("train_contours_from_trainX")
outdir.mkdir(exist_ok=True)

def safe_tricontour(x, y, z, title, filename, levels=20, min_pts=3):
    plt.figure(figsize=(6.4, 5.0))
    if len(x) >= min_pts:
        cn = plt.tricontourf(x, y, z, levels=levels)
        plt.colorbar(cn, label=title)
        plt.tricontour(x, y, z, levels=levels, linewidths=0.6)
    else:
        # fallback: tylko scatter gdy za mało punktów na kontur
        plt.scatter(x, y, s=8)
        plt.title(f"{title} (za mało punktów na kontur: n={len(x)})")
    # obrys domeny (opcjonalnie)
    plt.plot([-1, 1, 1, -1, -1], [-0.5, -0.5, 0.5, 0.5, -0.5], linewidth=1.0)
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlabel("x"); plt.ylabel("y"); plt.title(title)
    plt.tight_layout(); plt.savefig(filename, dpi=150); plt.close()

for t0 in picked_ts:
    mask = np.isclose(X[:,2], t0, atol=1e-12)
    X_t  = X[mask]
    if X_t.shape[0] == 0:
        continue

    # predykcja modelu na punktach treningowych dla tego t
    Y_t  = model.predict(X_t)  # (M,3): [ux,uy,F33]
    ux, uy = Y_t[:,0], Y_t[:,1]
    umag   = np.sqrt(ux**2 + uy**2)
    xg, yg = X_t[:,0], X_t[:,1]

    safe_tricontour(xg, yg, ux,   f"u_x @ t={t0:.6f}", outdir / f"ux_t{t0:.6f}.png")
    safe_tricontour(xg, yg, uy,   f"u_y @ t={t0:.6f}", outdir / f"uy_t{t0:.6f}.png")
    safe_tricontour(xg, yg, umag, f"|u| @ t={t0:.6f}", outdir / f"umag_t{t0:.6f}.png")

print("Gotowe.")


Wybrane czasy (najgęstsze): [np.float32(0.25), np.float32(0.5), np.float32(0.75), np.float32(0.0), np.float32(0.05)]  | liczności: [np.int64(681), np.int64(681), np.int64(681), np.int64(680), np.int64(680)]
Gotowe.
